In [1]:
import uuid 
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GoldVendas") \
    .config("spark.extraListeners", "io.openlineage.spark.agent.OpenLineageSparkListener") \
    .config("spark.openlineage.columnLineage.datasetLineageEnabled", "true") \
    .config("spark.openlineage.transport.type", "http") \
    .config("spark.openlineage.transport.url", "http://api:5000") \
    .config("spark.openlineage.namespace", "spark_integration") \
    .config("spark.openlineage.parentJobName", "customer-job") \
    .config("spark.openlineage.parentRunId", uuid.uuid4().__str__()) \
    .getOrCreate()

25/06/19 18:45:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
delta_path_vendas = "s3a://silver/fat_vendas/"
# Ler os dados
df_vendas = spark.read.format("delta").load(delta_path_vendas)

25/06/19 18:45:29 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [3]:
from pyspark.sql.functions import sum

# Agrupar por customer_name e somar a coluna price
df_total_por_cliente = df_vendas.groupBy("customer_name") \
                         .agg(sum("price").alias("total_price")) \
                         .orderBy("customer_name")

# Exibir o resultado
df_total_por_cliente.show()

25/06/19 18:45:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 9:>                                                          (0 + 2) / 2]

+-------------+------------------+
|customer_name|       total_price|
+-------------+------------------+
|          Ana| 473.3800048828125|
|       Carlos| 32.63999938964844|
|         João| 180.8699951171875|
|        Lucas|105.07999801635742|
|        Maria|214.82000160217285|
+-------------+------------------+



In [4]:
# Caminho para o bucket MinIO (formato S3)
delta_path = "s3a://gold/compras_cliente"

# Escrever como tabela Delta
df_total_por_cliente.write.format("delta").mode("overwrite").save(delta_path)

25/06/19 18:45:45 ERROR ContextFactory: Query execution is null: can't emit event for executionId 6
25/06/19 18:45:45 ERROR ContextFactory: Query execution is null: can't emit event for executionId 6
25/06/19 18:45:45 ERROR ContextFactory: Query execution is null: can't emit event for executionId 8
25/06/19 18:45:45 ERROR ContextFactory: Query execution is null: can't emit event for executionId 8
25/06/19 18:45:45 ERROR ContextFactory: Query execution is null: can't emit event for executionId 9
25/06/19 18:45:45 ERROR ContextFactory: Query execution is null: can't emit event for executionId 9
                                                                                

In [5]:
spark.stop()